In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df_MAIN = pd.read_csv("/Users/neelesh/Documents/FS-Sem2/fin_man/ESG-and-Financial-Performance/Data Collection/main_df_no_NAN_99p.csv")
df_MAIN_cleaned = df_MAIN.dropna()
df_MAIN_cleaned.describe()

,Unnamed: 0,Year,Total Return,ESG Score,ESG Combined Score,ESG Controversies Score,Social Pillar Score,Governance Pillar Score,Environmental Pillar Score,Revenue - Mean,Earnings Per Share - Mean,Return On Equity - Mean,Volume,Company Market Cap,EBITDA - Mean
count,50736.000000,50736.00000,50736.000000,50736.000000,50736.000000,50736.000000,50736.000000,50736.000000,50736.000000,5.073600e+04,50736.000000,50736.000000,5.073600e+04,5.073600e+04,5.073600e+04
mean,105943.495861,2014.95179,14.092822,45.176767,43.920164,92.278124,45.446937,50.942291,38.138041,3.252298e+11,94.246883,13.599591,1.164860e+06,3.290298e+11,4.537727e+10
std,59609.025312,4.98485,41.259729,19.707534,18.826651,20.426456,22.992075,21.653620,28.029774,1.564371e+12,715.321399,19.839521,4.793798e+06,1.664073e+12,2.360485e+11
min,2.000000,2002.00000,-76.348632,4.470837,4.470837,3.846154,1.611402,3.956835,0.000000,0.000000e+00,-92.500500,-129.960000,0.000000e+00,8.836997e+07,-5.121924e+08
25%,54743.750000,2011.00000,-11.440238,29.172295,28.908182,100.000000,26.996637,33.609719,12.249763,1.253814e+09,0.390475,6.599918,0.000000e+00,2.359950e+09,2.621201e+08
50%,106842.500000,2016.00000,9.430330,44.644679,43.401165,100.000000,44.240609,51.544241,37.116274,5.493699e+09,1.594085,12.002540,1.562000e+04,9.353239e+09,1.035209e+09
75%,156939.250000,2019.00000,32.696180,60.650268,58.300877,100.000000,63.518516,68.585440,61.739893,3.555839e+10,5.254563,19.373888,3.741870e+05,6.005076e+10,6.673981e+09
max,206912.000000,2022.00000,256.142197,89.721799,87.471019,100.000000,95.077388,94.714260,95.529762,2.770789e+13,15036.339140,182.646000,6.600124e+07,3.417197e+13,4.636861e+12


In [3]:
#   Columns in df_MAIN_cleaned

#   ['Year',
#  'ISIN',
#  'Total Return',
#  'ESG Score',
#  'ESG Combined Score',
#  'ESG Controversies Score',
#  'Social Pillar Score',
#  'Governance Pillar Score',
#  'Environmental Pillar Score',
#  'Revenue - Mean',
#  'Earnings Per Share - Mean',
#  'Return On Equity - Mean',
#  'Volume',
#  'Company Market Cap',
#  'EBITDA - Mean',
#  'Instrument',
#  'Country of Headquarters',
#  'Region of Headquarters',
#  'TRBC Industry Name',
#  'TRBC Industry Group Name',
#  'TRBC Business Sector Name',
#  'TRBC Economic Sector Name']

In [4]:
def model_ESG(n, conditioning_feature):
    
    results = pd.DataFrame({'number': n, 'mse': [0]*len(n), 'r2': [0]*len(n)})
    for i in range (len(n)):
        results['number'][i] = n[i]
        top_n_df = df_MAIN_cleaned.groupby('Region of Headquarters').apply(lambda x: x.nlargest(n[i], conditioning_feature)).reset_index(drop=True)
        top_n_df_reg = top_n_df[['Total Return',
                             'Revenue - Mean',
                             'Earnings Per Share - Mean',
                             'Return On Equity - Mean',
                             'Volume',
                             'Company Market Cap',
                             'EBITDA - Mean',
                             'ESG Score']]
        X = top_n_df_reg.drop('ESG Score',axis =1)
        y = top_n_df_reg['ESG Score']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 101)
        regr = RandomForestRegressor(n_estimators = 10, max_depth = 10, random_state = 101)
        regr.fit(X_train, y_train.values.ravel())
        predictions = regr.predict(X_test)
        # Mean squared error (MSE)
        results['mse'][i] = mean_squared_error(y_test.values.ravel(), predictions)
        # R-squared scores
        results['r2'][i] = r2_score(y_test.values.ravel(), predictions)
        
    return results
    

In [5]:
n = [5, 10, 20, 50, 100, 150, 200, 250]
conditioning_feature = 'Company Market Cap'

results_ = model_ESG(n, conditioning_feature)
results_

/var/folders/wx/1_76tj0s15gc4yxmndvw4l_00000gn/T/ipykernel_64514/3639219928.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['r2'][i] = r2_score(y_test.values.ravel(), predictions)
/var/folders/wx/1_76tj0s15gc4yxmndvw4l_00000gn/T/ipykernel_64514/3639219928.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['number'][i] = n[i]
/var/folders/wx/1_76tj0s15gc4yxmndvw4l_00000gn/T/ipykernel_64514/3639219928.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

,number,mse,r2
0,5,128.315233,0.595515
1,10,99.655351,0.811140
2,20,270.746706,0.343665
3,50,152.065144,0.537682
4,100,238.698296,0.423042
5,150,218.153326,0.398164
6,200,300.603727,0.270050
7,250,256.749100,0.188513
